In [1]:
import numpy as np
import numpy.linalg as lin

np.set_printoptions(precision=3, linewidth=150, suppress=True)

## Systèmes matriciels

Un système de plusieurs équations à autant d'inconnues peut s'écrire sous forme d'un système matriciel de la forme $A {\bf x} = {\bf b}$ :

$$
\begin{bmatrix}
a_{11} a_{12} \ldots a_{1n} \\
a_{21} a_{22} \ldots a_{2n} \\
 \vdots \\
a_{n1} a_{n2} \ldots a_{nn} \\
\end{bmatrix}
\;
\begin{bmatrix}
x_{1} \\
x_{2} \\
\vdots \\
x_{n} \\
\end{bmatrix}
=
\begin{bmatrix}
b_{1} \\
b_{2} \\
\vdots \\
b_{n} \\
\end{bmatrix}
$$

Ainsi si votre épicier ne vous donne pas le prix des pommes, poires et bananes mais que vous en achetez 3 fois des quantités différentes, vous pouvez retrouver le prix de chaque fruit. Il suffit de résoudre les 3 équations
qui correspondent aux 3 achats ou le système matriciel 3x3 correpondant. Pour résourdre ce dernier la facon intuitive est d'inverser A  et de calculer $A^{-1}\, {\bf b}$ :

In [4]:
# A est la quantité de chaque fruit achetée
# x est le prix de chaque fruit
# b est la somme qu'on a payé pour chaque course

A = np.array([[6,5,4], [5,3,2], [7,3,2]])
b = np.array([11.7, 7.9, 9.5])
print(A, b)

x = lin.inv(A) @ b
x

[[6 5 4]
 [5 3 2]
 [7 3 2]] [11.7  7.9  9.5]


array([0.8, 0.9, 0.6])

Les systèmes matriciels sont probablement l'application la plus importante des matrices car quoi de plus important que de connaitre le prix des bananes ?

## Résolution d'un système matriciel

On regardera les méthodes suivantes :

* Méthode du pivot de Gauss (rendre la matrice triangulaire)
* Décomposition LU (effet de bord de la résolution par le pivot de Gauss)
* Méthode de Gauss-Jordan (rendre la matrice diagonale)

**Note :** Si vous trouvez la suite difficile, faites une pause et regardez cette [explication sur une matrice 3x3](https://www.youtube.com/watch?v=VoMsTiJOUE4). Elle commence par triangulariser la matrice comme le fait
la méthode du pivot de Gauss, puis elle continue pour diagonaliser la matrice comme le fait Gauss-Jordan et
finalement elle arrive à l'inverse de la matrice. C'est très bien pour comprendre ou faire les calculs à la main
sur une petite matrice, c'est horrible à programmer. Ce que je vous montre c'est la transformation de l'intuition
qu'offre cette vidéo en un calcul matriciel rapide et simple à programmer.

### Méthode du pivot de Gauss

On transforme A en une matrice triangulaire qui permet ensuite de résoudre le système en O(n²) opérations.

Pour cela on commence à mettre des 0 sur la première colonne en dessous de la diagonale. Pour cela il
suffit de multiplier A par la matrice $E_1$ suivante, *à gauche* :


$$
E_1 = 
\begin{bmatrix}
\;1 \quad 0\; 0 \ldots 0 \\
\frac{-a_{21}}{a_{11}} \, 1\; 0  \ldots 0 \\
\frac{-a_{31}}{a_{11}} \, 0\; 1  \ldots 0 \\
\vdots \\
\frac{-a_{n1}}{a_{11}}\; 0\; 0  \ldots 1 \\
\end{bmatrix}
$$

$E_2$ sera la matrice équivalente avec des termes $\frac{-a_{k2}}{a_{22}}$ sous la diagonales afin des 0 dans la 2e colonne de A sous la diagonale, etc.

Si vous ne connaissez pas cette méthode, je vous conseille de vérifier à la main sur une matrice 4x4 que
$E_1 \, A$ met bien des zéros là où il faut. Ensuite faites le travail sur la 2e colonne.

Bien sur si on multiplie A par $E_1$, il faut aussi multiplier ${\bf b}$ par $E_1$ pour que le système matriciel soit
équivalent. Cette multiplication $E_1 \, \bf b$ peut se calculer nettement plus rapidement qu'avec un 
produit matrice vecteur. Donnez la bonne formule et vérifiez dans le code ci-dessous.

#### Système matriciel avec matrice triangulaire

Pour finir il faut résoudre U x = c avec U une matrice triangulaire supérieur. Cela se fait en partant
de la dernière ligne qui donne directement `x[-1] = c[-1] / U[-1,-1]`. Une fois `x[-1]` connu, on en
déduit la valeur de `x[-2]` avec l'avant dernière ligne, etc.

En faisant les calculs on constate qu'il faut environ n²/2 additions et multiplications.

Là encore, si vous ne vous rappelez pas comment on fait, faites le à la main avec une matrice 3x3. Vous pouvez aussi ajouter des `print` dans le code qui suit pour voir les matrices intermédiaires.

In [5]:
# Cf analyse du code ci-dessous

def solve_gauss(A, b):
    A = A.astype(np.float64)   # si A a des entiers, on va avoir des erreurs de calculs
    for i in range(len(A)-1):
        coefs = - A[i+1:,i] / A[i,i]  # Normalement il faut tester que A[i,i] != 0
        A[i+1:, i:] += np.outer(coefs, A[i, i:])
        b[i+1:] += coefs * b[i]
    # A est maintenant triangulaire supérieur
    res = np.zeros(len(b), dtype=b.dtype)
    res[-1] = b[-1] / A[-1,-1]
    for i in range(len(A)-1)[::-1]:
        res[i] = (b[i] - A[i,i+1:] @ res[i+1:]) / A[i,i]
    return res


#### Analyse du code

Comme **E** = **Id** + **Coefs**, on a **E A** = **A** + **Coefs A** et donc **A** += **Coefs A**. Ici **Coefs** est la matrice **E** ci-dessus sans les 1 sur la diagonale.

Le problème est que `Coefs @ A` fait n³ opérations ce qui est beaucoup trop et surtout inutile sachant que **Coefs** est un vecteur dans une matrice de zéros. Si ce vecteur est dans la colonne $i$ alors seuls les termes de la ligne $i$ de A serviront pour générer ce produit matriciel. De plus on a `res[j,k] = Coef[j,i] * A[i,k]` ce qui s'écrit en Numpy `res = np.outer(coefs, A[i,:])` si `coefs` est la colonne $i$ de `Coefs`.

Comme la matrice **Coefs** n'a des valeurs non nulles qu'en dessous de la diagonale, cela veut dire que le résultat `res` de la multiplication n'a que des zéros au dessus de la ligne $i$. Donc autant ne faire que le calcul pour ce qui est en
dessus de la ligne $i$ : `A[i+1:, i:] += np.outer(coefs, A[i, i:])` avec `coefs` un vecteur de taille $n-i$. 

In [6]:
A = 10 * np.random.random(size=(5,5))
b = A.sum(axis=1)
print(f"A\n {A} \nb\n {b}")
solve_gauss(A, b)

A
 [[3.23  6.494 1.931 5.034 6.7  ]
 [0.724 4.169 0.105 7.029 5.157]
 [9.668 8.432 6.053 9.566 7.324]
 [1.499 6.048 2.711 7.883 9.19 ]
 [0.575 6.654 6.248 6.431 4.692]] 
b
 [23.389 17.183 41.044 27.332 24.599]


array([1., 1., 1., 1., 1.])

### Complexité du pivot de Gauss

#### Exercice

Calculer la complexité de `solve_gauss`. Vous devez trouver n³/3 multiplications et autant d'additions plus des termes de degrés inférieurs.

### Décomposition LU (Lower, Upper)

Si on a besoin de résoudre plusieurs systèmes matriciels avec A, alors il est préférable de décomposer
A en un produit d'une matriciel triangulaire inférieur et d'une matrice triangulaire supérieure :

$A = L\, U$

Pour cela il suffit de relancer le pivot de Gauss mais au lieu de modifier **b** à chaque itération, on
fabrique à la volée l'inverse de la matrice $E_n \ldots E_2\, E_1$ : 

$$
E_n \ldots E_2\, E_1 \, A\, x = E_n \ldots E_2\, E_1 \, b \quad \textrm{donc} \\
(E_n \ldots E_2\, E_1)^{-1} \, E_n \ldots E_2\, E_1 \, A\, x =  b \\
E_1^{-1} \, E_2^{-1} \ldots E_n^{-1} \; E_n \ldots E_2\, E_1 \, A\, x =  b \\
$$

Ce calcul est très simple car ces matrices inverses ont la
même forme que les $E_k$ avec simplement le valeurs opposées sous la diagonale :

$$
E_1^{-1} = 
\begin{bmatrix}
\;1 \quad 0\; 0 \ldots 0 \\
\frac{a_{21}}{a_{11}} \, 1\; 0  \ldots 0 \\
\frac{a_{31}}{a_{11}} \, 0\; 1  \ldots 0 \\
\vdots \\
\frac{a_{n1}}{a_{11}}\; 0\; 0  \ldots 1 \\
\end{bmatrix}
$$

et le produit $E^{-1} = E_1^{-1} \,E_1^{-1} \,E_2^{-1} \,E_3^{-1} \, \ldots \,E_n^{-1}$ est simplement la concaténation
des colonnes (je vous laisse vérifier) :

$$
E^{-1} = 
\begin{bmatrix}
\;1 \quad 0\; \; 0 \ldots 0 \\
\frac{a_{21}}{a_{11}} \; \; 1\; \; 0  \ldots 0 \\
\frac{a_{31}}{a_{11}} \, \frac{a_{32}}{a_{22}}  \; 1  \ldots 0 \\
\vdots \\
\frac{a_{n1}}{a_{11}}\; \frac{a_{n2}}{a_{22}}\; \frac{a_{n3}}{a_{33}}  \ldots 1 \\
\end{bmatrix}
$$

et ainsi on a $L = E^{-1}$ et $U = E_n \ldots E_2\, E_1 \, A$ puisque tout le travail fait sur A est
justement pour mettre des 0 sous la diagonale et donc en faire une matrice triangulaire supérieure.

Une fois la décomposition LU faites, résoudre $L\, U \, {\bf x} = {\bf b}$ se fait en 2 étapes :

* on résoud $L\, {\bf y}  = {\bf b}$ ce qui donne ${\bf y}$ 
* puis on résoud $U\, {\bf x} = {\bf y}$ et on a notre solution ${\bf x}$

en O(n²) opérations à chaque fois puisqu'il s'agit de matrices triangulaires.

In [8]:
def LU(A):
    L = np.diag([1.,] * len(A))
    for i in range(len(A)-1):
        E = np.diag([1.,] * len(A))
        E[i+1:,i] = - A[i+1:,i] / A[i,i]
        L[i+1:,i] = -E[i+1:,i]
        A[i:, i:] = E[i:,i:] @ A[i:,i:]
    return L, A

In [10]:
A = 10 * np.random.random(size=(5,5))
print("A\n",A)
L,U = LU(A.copy())  # Attention, notre fonction modifie A
                    # si on veut le réutiliser il faut une copie
print(f"L\n{L}\nU\n{U}")
A - (L @ U)

A
 [[8.486 7.849 4.558 6.688 3.333]
 [6.786 4.237 4.266 1.415 3.992]
 [7.59  0.276 1.563 5.598 1.948]
 [8.96  4.474 2.436 1.099 5.844]
 [6.426 7.346 9.967 2.491 4.595]]
L
[[ 1.     0.     0.     0.     0.   ]
 [ 0.8    1.     0.     0.     0.   ]
 [ 0.894  3.307  1.     0.     0.   ]
 [ 1.056  1.87   0.775  1.     0.   ]
 [ 0.757 -0.688 -1.519 -1.658  1.   ]]
U
[[ 8.486  7.849  4.558  6.688  3.333]
 [ 0.    -2.039  0.621 -3.933  1.327]
 [ 0.     0.    -4.569 12.624 -5.421]
 [ 0.     0.     0.    -8.385  4.043]
 [ 0.     0.     0.     0.     1.449]]


array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

### Gauss Jordan

L'idée est de diagonaliser A par des multiplications matricielles similaires à celles de Gauss mais qui
annulent aussi les termes au dessus de la diagonale :

$$
E_3 = 
\begin{bmatrix}
1 \; 0\; \frac{-a_{13}}{a_{33}} \;  0 \ldots 0 \\
0 \; 1\; \frac{-a_{23}}{a_{33}} \;  0 \ldots 0 \\
0 \; 0\quad 1 \quad  0 \ldots 0 \\
0 \; 0\; \frac{-a_{43}}{a_{33}}  1 \ldots 0 \\
\vdots \\
0 \; 0\; \frac{-a_{n3}}{a_{33}}  0 \ldots 1 \\
\end{bmatrix}
$$


Cette méthode est plus compacte à écrire que Gauss car il n'y a pas de remonté finale, mais elle est plus lente car on modifie tout A et b à chaque itération.

In [12]:
def solve_gauss_jordan(A,b):
    for i in range(len(A)):
        d1 = np.diag([1.,] * len(A))
        d1[:,i] = - A[:,i] / A[i,i]
        A = d1 @ A
        b = d1 @ b
    return b / np.diag(A)

In [13]:
A = 10 * np.random.random(size=(5,5))
b = A.sum(axis=1)
solve_gauss_jordan(A, b)

array([1., 1., 1., 1., 1.])

### Comparaison de la vitesse de méthodes

In [18]:
n = 200
A = 10 * np.random.random(size=(n,n))
b = A.sum(axis=1)

In [19]:
%timeit solve_gauss_jordan(A, b)

729 ms ± 8.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%timeit solve_gauss(A, b)   # beaucoup plus rapide que Gauss-Jordan

4.81 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
%timeit B = lin.inv(A)  # encore plus rapide car numpy native

3.75 ms ± 91 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
%timeit lin.solve(A, b) # un peu plus rapide

1.29 ms ± 35.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Pour n=5000, `lin.solve` est 3 fois plus rapide que `lin.inv` sur ma machine. Et plus on augmente $n$ et plus
l'écart se creuse 

**Note**: Les calculs utilisent tous les processeurs de ma machine, Numpy sait paralléliser et utilise
les bonnes bibliothèques (BLAS et Lapack) s'il est bien installé.

#### Morale

* Une bonne méthode est mieux qu'une mauvaise (et oui !).
* Une bonne bibliothèque fait toute la différence (vous ne ferez pas mieux que BLAS et Lapack).

## Erreurs d'arrondi

In [25]:
np.set_printoptions(precision=20, linewidth=150, suppress=True)

np.finfo(np.float32)

finfo(resolution=1e-06, min=-3.4028235e+38, max=3.4028235e+38, dtype=float32)

In [26]:
e = 1E-6
A = np.array([[e, 1], [1, 2]], dtype='float32')
b = np.array([1., 3.], dtype='float32')
print(f"A\n {A} \nb\n {b}")

A
 [[0.000001 1.      ]
 [1.       2.      ]] 
b
 [1. 3.]


In [27]:
x = solve_gauss(A.copy(),b)
print("x =", x)
A @ x                # on ne retrouve pas exactement b

x = [1.013279 0.999999]


array([1.      , 3.013277], dtype=float32)

Il y a un problème d'arrondi évident sachant que la solution est [1 + 2*e, 1 - e] :

In [28]:
x = np.array([1 + 2*e, 1 - e], dtype='float32')
print("La solution est :", x)
A @ x             # on retrouve b

La solution est : [1.000002 0.999999]


array([1., 3.], dtype=float32)

Le problème est qu'on divise les valeurs de la première colonne de A par le premier pivot : 1E-6.

Aussi on multiplie par 1E6 ce qui va faire des grandes valeurs or la précision est au niveau du 6e chiffre,
on pert les suivants comme le montre la simple addition qui suit :

In [29]:
x = np.array([1E6], dtype='float32')   # 1 million
y = np.array([1E-2], dtype='float32')  # 0.01
x+y                                    # résultat : 1 million sans rien après la virgule

array([1000000.], dtype=float32)

**À mélanger des grands nombres avec des petits, on fait des erreurs d'arrondi.**

C'est vrai pour les calculs matriciels qu'on fait, c'est toujours vrai et c'est pour cela qu'on aime normaliser
les données sur lesquelles on travaille lorsqu'on le peut.

### Solution au problème d'arrondi dans le cas du pivot de Gauss

Il suffit d'échanger les lignes (ou colonnes mais c'est plus compliqué) pour avoir la plus grande valeur,
en valeur absolue, comme pivot. C'est la méthode du pivot partiel.

La méthode du pivot total consiste à prendre comme pivot la plus grande valeur parmis la ligne et la colonne concernées.
